In [2]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as Logit 
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [65]:
#m é a confusion matrix, ind é o indíce da variável que queremos testar
#ind começa em 0
def sensitivity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    x=m[ind][ind]
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[n][ind]]
            n+=1
    y=sum(q)
    return (x/(x+y))

def specificity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[ind][n]]
            n+=1
    x=sum(q)
    f=[]
    i=0
    while i<len(m):
        c=0
        if i==ind:
            i+=1
        else:
            while c<len(m):
                if c==ind:
                    c+=1
                else:
                    f+=[m[i][c]]
                    c+=1
            i+=1
    y=sum(f)
    #print(q,f)  #verifica se os valores q estamos a escolher são os certos
    return (y/(x+y))

#A percentagem de previsões corretas feitas pelo modelo
#i é a linha da confusion matrix (o outcome que queremos ver a percentagem de previsões corretas)
def pred_percent(m,i):
    #print(m)
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    #print('O índice começa em 0')
    x=m[i][i]
    soma=sum(m[i])
    if soma==0:
        return 'Não previu nada para esta variável'
    else:
        return round((x/soma)*100),'%'

In [243]:
#faz a conf matrix do model (já tem de estar fit())
#o modelo tem de ter a probabilidade ligada
def conf_matrix(model,X_test,y_test,limite=0.5):
    prob=model.predict_proba(X_test)
    Y=list(y_test)
    cmat=np.array([[0,0],[0,0]])   #confusion matrix
    x=0
    y=0
    l=0
    for i in range(len(prob)):
        
        if prob[i][1]>limite:
            print(prob[i],i)
            what=1
        else:
            what=0
        if what==1 and Y[i]==1: #previsão 1 e realidade 1
            x+=X_test.iloc[i]['LowODD']
            l+=1
            cmat[0][0]+=1
            #print((X_test[i:i+1]))
            print(i,'|',X_test.iloc[i]['LowODD'])
        elif what==1 and Y[i]==0: #previsão 1 e realidade 0
            y+=X_test.iloc[i]['LowODD']
            l+=1
            cmat[0][1]+=1
            #print((X_test[i:i+1]),'enganou-se')
            print(i,'|',X_test.iloc[i]['LowODD'],'wrooooooooooooong')
        elif what==0 and Y[i]==1:  #previsão 0 e realidade 1
            cmat[1][0]+=1
        elif what==0 and Y[i]==0:  #previsão 0 e realidade 0
            cmat[1][1]+=1
        else:
            print('Erro em conf_matrix')
    print(l)
    if l>0:
        print((x+y)/l)
    return (cmat,
           pred_percent(cmat,0))



_______________________________________________

In [47]:
len(biglion.index)

1048

In [3]:
biglion=pd.read_excel('BigLion.xlsx',0)
Xbiglion=biglion.drop(['Div','Date','Time','LO_Team','HO_Team','Won/NotWon',],axis=1)
Ybiglion=biglion['Won/NotWon']
#X_bigtrain=pd.concat([Xbiglion[:700],Xbiglion[740:]],sort=False)
#y_bigtrain=pd.concat([Ybiglion[:700],Ybiglion[740:]],sort=False)
X_bigtrain=Xbiglion[:1010]
y_bigtrain=Ybiglion[:1010]
X_bigtest=Xbiglion[1010:]
y_bigtest=Ybiglion[1010:]

In [4]:
Mbig1=SVC(C=0.0001,kernel='rbf',probability=True)
Mbig1.fit(X_bigtrain,y_bigtrain)

C:\Users\joaom\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [5]:
Mbig2=Logit(C=0.0005)
Mbig2.fit(X_bigtrain,y_bigtrain)

C:\Users\joaom\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.0005, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
Mbig3=KMeans(n_clusters=2)
y_predicted=Mbig3.fit_predict(X_bigtest)
y_predicted

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1])

In [12]:
list(y_bigtest)

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1]

In [256]:
Modd1.predict_proba([[10,3,3,1.15,6.95,14.07,1.6,2.12,15,0,10.4,10.6,4.4,4.4,1.6,
                2,1.333,1,1,1,1,7,1,-18,10,9,4,4,1,2,0,1,10,0,13]])

array([[0.08905263, 0.91094737]])

In [244]:
conf_matrix(Mbig1,X_bigtest,y_bigtest,limite=0.5)

[0.48020296 0.51979704] 0
0 | 2.69
[0.49421776 0.50578224] 3
3 | 2.38
[0.22305788 0.77694212] 34
34 | 2.24
3
2.436666666666667


(array([[ 3,  0],
        [21, 12]]), (100.0, '%'))

In [245]:
conf_matrix(Mbig2,X_bigtest,y_bigtest,limite=0.5)

[0.34066358 0.65933642] 0
0 | 2.69
[0.26707404 0.73292596] 1
1 | 2.18
[0.21755485 0.78244515] 2
2 | 1.36
[0.29304897 0.70695103] 6
6 | 2.09 wrooooooooooooong
[0.4881345 0.5118655] 7
7 | 1.82
[0.45952375 0.54047625] 8
8 | 1.85 wrooooooooooooong
[0.21716474 0.78283526] 9
9 | 1.33 wrooooooooooooong
[0.37496866 0.62503134] 12
12 | 2.79
[0.4807712 0.5192288] 13
13 | 1.68 wrooooooooooooong
[0.46172089 0.53827911] 16
16 | 1.87
[0.11110225 0.88889775] 17
17 | 1.09
[0.37053841 0.62946159] 19
19 | 2.43
[0.26250667 0.73749333] 22
22 | 1.64 wrooooooooooooong
[0.39842668 0.60157332] 23
23 | 2.56
[0.47577204 0.52422796] 24
24 | 2.04 wrooooooooooooong
[0.31074917 0.68925083] 27
27 | 1.26
[0.39375072 0.60624928] 28
28 | 2.27
[0.30599628 0.69400372] 29
29 | 1.77 wrooooooooooooong
[0.43744616 0.56255384] 30
30 | 2.14
[0.28885205 0.71114795] 31
31 | 1.66
[0.48276493 0.51723507] 32
32 | 1.78
[0.36389826 0.63610174] 33
33 | 1.38
[0.36206761 0.63793239] 34
34 | 2.24
[0.4488969 0.5511031] 35
35 | 1.78
24
1.9

(array([[17,  7],
        [ 7,  5]]), (71.0, '%'))

In [160]:
biglion

,Div,Date,Time,LO_Team,HO_Team,H/A,L_AwayLast_5,H_AwayLast_5,LowODD,DrawODD,HighODD,L_LowestLost,H_LowestLost,L_Points_5,H_Points_5,LoS5,HoS5,LoST5,HoST5,LoG5,HoG5,LoConceedAver_3,HoConceedAver_3,Lo_WinCount_,Ho_WinCount_,Lo_ScoresCount,Ho_ScoresCount,Lo_CleanSheet,Ho_CleanSheet,LoS,HoS,LoST,HoST,LoG,HoG,LoConcd,HoConcd,OddCounter,SurprCount_1.4,Won/NotWon
0,I1,2019-10-20,17:00,Parma,Genoa,10,2,2,2.51,3.21,2.95,2.28,2.37,6,1,8.6,8.6,4.4,4.4,1.0,1.0,1.000000,2.000000,0,0,0,1,0,0,5,11,2,6,0,1,1,2,-1,100,1
1,D1,2019-10-20,17:00:00,Schalke 04,Hoffenheim,0,2,3,2.55,3.52,2.69,1.64,1.69,13,5,14.8,14.8,6.0,6.0,2.8,2.8,1.000000,1.666667,0,1,5,1,0,0,15,8,3,5,1,2,1,1,2,100,0
2,SP1,2019-10-20,17:30,Ath Bilbao,Valladolid,10,3,2,1.69,3.46,6.03,2.16,2.23,5,6,10.6,10.6,3.2,3.2,0.6,0.6,1.000000,0.333333,0,0,0,0,0,2,6,5,2,1,0,0,1,0,-1,100,0
3,I1,2019-10-20,19:45,Milan,Lecce,10,3,3,1.40,4.82,7.90,2.06,1.92,6,6,10.0,10.0,4.6,4.6,1.0,1.0,2.000000,1.666667,1,0,3,1,0,0,9,5,7,2,2,1,1,3,5,100,0
4,SP1,2019-10-20,20:00,Sevilla,Levante,10,3,3,1.33,5.53,8.64,1.55,2.15,6,5,14.4,14.4,4.6,4.6,1.2,1.2,3.000000,1.666667,0,1,0,3,0,0,16,9,4,2,0,2,4,1,2,3,1
5,F1,2019-10-20,20:00,Marseille,Strasbourg,10,3,3,1.85,3.44,4.60,1.65,2.05,6,6,15.2,15.2,4.4,4.4,1.4,1.4,1.333333,1.000000,0,0,2,0,0,0,13,15,4,2,1,0,3,1,5,100,1
6,I1,2019-10-21,19:45,Fiorentina,Brescia,0,2,3,2.00,3.57,3.70,2.71,3.04,11,3,10.8,10.8,6.2,6.2,1.6,1.6,0.666667,1.333333,3,0,4,4,1,0,8,7,4,3,1,1,0,2,-1,100,1
7,E0,2019-10-21,20:00:00,Arsenal,Sheffield United,0,2,3,1.93,3.79,3.88,2.01,2.46,9,5,15.8,15.8,5.0,5.0,1.8,1.8,1.000000,0.333333,1,0,8,0,1,1,12,9,2,3,1,0,0,0,1,100,0
8,D1,2019-10-25,19:30:00,Mainz,FC Koln,10,3,2,2.55,3.54,2.70,2.65,2.15,6,4,13.6,13.6,4.2,4.2,1.0,1.0,1.000000,1.666667,0,1,0,2,0,1,7,16,1,6,0,3,1,0,-1,100,1
9,I1,2019-10-25,19:45,Verona,Sassuolo,10,3,2,2.44,3.27,3.02,2.56,2.70,5,3,9.4,9.4,5.4,5.4,0.8,0.8,1.000000,3.000000,0,0,0,2,0,0,14,7,6,4,0,3,2,4,6,100,0


In [210]:
bigOdd=pd.read_excel('BigLion.xlsx',2)
XbigOdd=bigOdd.drop(['Div','Date','Time','LO_Team','HO_Team','Won/NotWon',],axis=1)
YbigOdd=bigOdd['Won/NotWon']
X_bigtrain=pd.concat([XbigOdd[:22],XbigOdd[23:]],sort=False)
y_bigtrain=pd.concat([YbigOdd[:22],YbigOdd[23:]],sort=False)
#X_oddtrain=XbigOdd[:31]
#y_oddtrain=YbigOdd[:31]
X_oddtest=XbigOdd.iloc[[22]]
y_oddtest=YbigOdd.iloc[[22]]

In [227]:
Modd1=SVC(C=1,kernel='rbf',probability=True)
Modd1.fit(X_oddtrain,y_oddtrain)

C:\Users\joaom\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [212]:
Modd2=Logit(C=0.0000001)
Modd2.fit(X_oddtrain,y_oddtrain)

C:\Users\joaom\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1e-07, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [209]:
conf_matrix(Modd2,X_oddtest,y_oddtest,limite=0.5)

[0.4995686 0.5004314] 0
0 | 1.17 wrooooooooooooong
1
1.17


(array([[0, 1],
        [0, 0]]), (0.0, '%'))

In [221]:
conf_matrix(Modd1,X_oddtest,y_oddtest,limite=0.5)

[0.03299957 0.96700043] 0
0 | 1.19
1
1.19


(array([[1, 0],
        [0, 0]]), (100.0, '%'))

In [205]:
Modd2.predict([[10,3,3,1.15,6.95,14.07,1.6,2.12,9,8,10.4,10.6,4.4,4.4,1.6,
                2,1.333,1,1,1,1,7,1,-18,10,9,4,4,1,2,0,1,13,0,10]])

array([1], dtype=int64)

In [220]:
Modd2.predict_proba([[10,3,3,1.15,6.95,14.07,1.6,2.12,9,8,10.4,10.6,4.4,4.4,1.6,
                2,1.333,1,1,1,1,7,1,-18,10,9,4,4,1,2,0,1,13,0,10]])

array([[0.49830198, 0.50169802]])

In [233]:
Modd2.predict_proba([[10,3,3,212.15,6.95,14.07,-1.6,-2.12,90,8,10.40,100.6,4.4,4.4,3.6,
                2,10.333,10,10,-1,-1,-7,-1,-18,-101,91,42,421,112,212,110,5,-2222222,0,-222222]])

array([[9.99954219e-01, 4.57807499e-05]])

In [ ]:
Modd2